# Clique Cover

In graph theory, a clique cover or partition into cliques of a given undirected graph is a partition of the vertices of the graph into cliques, subsets of vertices within which every two vertices are adjacent. A minimum clique cover is a clique cover that uses as few cliques as possible. The minimum k for which a clique cover exists is called the clique cover number of the given graph.

https://en.wikipedia.org/wiki/Clique_cover

# Cost Function
When we color vertex $v$ with color $i$ we show using qubit as $x_{v,i}$.
Now we have the cost function

$ \displaystyle H = A \sum_v \left( 1 - \sum_{i = 1}^n x_{v,i} \right)^2 + B \sum_{i=1}^n \left[ \frac {1}{2} \left( -1 + \sum_v x_{v,i} \right) \sum_v x_{v,i} - \sum_{(uv) \in E} x_{u,i}x_{v.i} \right]$

Expand it and we have,

$ \displaystyle H = A \sum_v \left\{ -2 \sum_{i=1}^n x_{v,i} + \left(\sum_{i=1}^n x_{v,i}\right)^2 \right\} + B \sum_{i=1}^n \left\{ -\frac{1}{2} \sum_v x_{v,i} + \frac{1}{2}\left( \sum_v x_{v,i}\right)^2 - \sum_{(u,v) \in E} x_{u,i}x_{v,i}\right\}+ Const. $  
$ \displaystyle = A \sum_v \left( -2 \sum_{i=1}^n x_{v,i} + \sum_{i=1}^n x_{v,i}^2 + 2\mathop{ \sum \sum }_{i \neq j }^{n} x_{v,i}x_{v,j} \right) + B \sum_{i=1}^n \left\{ \frac{1}{2} \left(-\sum_v x_{v,i} + \sum_v x_{v,i}^2 + \mathop{\sum \sum}_{u \neq v}^{n} x_{u,i}x_{v,i} \right) - \sum_{(u,v) \in E} x_{u,i}x_{v,i}\right\}+ Const. $  
$ \displaystyle = A \sum_v \left( - \sum_{i=1}^n x_{v,i}^2 + 2\mathop { \sum \sum }_{i \neq j }^{n} x_{v,i}x_{v,j} \right) + B \sum_{i=1}^n \left( \frac{1}{2} \mathop{\sum \sum}_{u \neq v}^{n}x_{u,i}x_{v,i} - \sum_{(u,v) \in E} x_{u,i}x_{v,i}\right)+ Const. $

# Solving QUBO

In [0]:
!pip install blueqat

In [0]:
import blueqat.opt as wq
import numpy as np

def get_qubo(adjacency_matrix, n_color, A, B):
    graph_size = len(adjacency_matrix)
    qubo_size = graph_size * n_color
    qubo = np.zeros((qubo_size, qubo_size))
    indices = [(u,v,i,j) for u in range(graph_size) for v in range(graph_size) for i in range(n_color) for j in range(n_color)]
    for u,v,i,j in indices:
        ui = u * n_color + i
        vj = v * n_color + j
        if ui > vj:
            continue
            
        if ui == vj:
            qubo[ui][vj] -= A
        if u == v and i != j:
            qubo[ui][vj] += A * 2
        if u != v and i == j:
            qubo[ui][vj] += B * 0.5
            if adjacency_matrix[u][v] > 0:
                qubo[ui][vj] -= B
    return qubo

In [0]:
def show_answer(q, graph_size, n_color):
    print(q)
    for v in range(graph_size):
        color = []
        for i in range(n_color):
            index = v * n_color + i
            if q[index] > 0:
                color.append(i)
        print(f"vertex{v}'s color is {color}")

In [0]:
def calculate_H(q, adjacency_matrix, n_color, A, B):
    graph_size = len(adjacency_matrix)
    h_a = calculate_H_A(q, graph_size, n_color, A)
    h_b = calculate_H_B(q, adjacency_matrix, n_color, B)
    print(f"H = {h_a + h_b}")
    return h_a + h_b

def calculate_H_A(q, graph_size, n_color, A):
    hamiltonian = 0
    for v in range(graph_size):
        sum_x = 0
        for i in range(n_color):
            index = v * n_color + i
            sum_x += q[index]
        hamiltonian += (1 - sum_x) ** 2
    hamiltonian *= A
    print(f"H_A = {hamiltonian}")
    return hamiltonian

def calculate_H_B(q, adjacency_matrix, n_color, B):
    graph_size = len(adjacency_matrix)
    hamiltonian = 0
    for i in range(n_color):
        sum_x = 0
        for v in range(graph_size):
            vi = v * n_color + i
            sum_x += q[vi]
            for u in range(graph_size):
                if u >= v:
                    continue
                ui = u * n_color + i
                hamiltonian -= adjacency_matrix[u][v] * q[ui] * q[vi]
        hamiltonian += 0.5 * (-1 + sum_x) * sum_x
    hamiltonian *= B
    print(f"H_B = {hamiltonian}")
    return hamiltonian

This time we have an example like below,

![005.png](https://github.com/mdrft/Wildqat/blob/master/examples_ja/img/005.png?raw=1)

In [5]:
adjacency_matrix = \
[ \
    [0,1,1,0,0], \
    [1,0,1,1,1], \
    [1,1,0,1,0], \
    [0,1,1,0,1], \
    [0,1,0,1,0], \
]

n_color = 2
A = 0.1
B = 0.1

annealer = wq.opt()
annealer.qubo = get_qubo(adjacency_matrix, n_color, A, B)
for _ in range(10):
    q = annealer.sa()
    calculate_H(q, adjacency_matrix, n_color, A, B)
    show_answer(q, len(adjacency_matrix), n_color)
    print()

H_A = 0.1
H_B = 0.2
H = 0.30000000000000004
[1, 0, 1, 0, 1, 1, 1, 0, 0, 1]
vertex0's color is [0]
vertex1's color is [0]
vertex2's color is [0, 1]
vertex3's color is [0]
vertex4's color is [1]

H_A = 0.0
H_B = 0.0
H = 0.0
[1, 0, 1, 0, 1, 0, 0, 1, 0, 1]
vertex0's color is [0]
vertex1's color is [0]
vertex2's color is [0]
vertex3's color is [1]
vertex4's color is [1]

H_A = 0.0
H_B = 0.1
H = 0.1
[0, 1, 1, 0, 1, 0, 1, 0, 1, 0]
vertex0's color is [1]
vertex1's color is [0]
vertex2's color is [0]
vertex3's color is [0]
vertex4's color is [0]

H_A = 0.0
H_B = 0.1
H = 0.1
[1, 0, 0, 1, 0, 1, 0, 1, 0, 1]
vertex0's color is [0]
vertex1's color is [1]
vertex2's color is [1]
vertex3's color is [1]
vertex4's color is [1]

H_A = 0.0
H_B = 0.30000000000000004
H = 0.30000000000000004
[0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
vertex0's color is [1]
vertex1's color is [1]
vertex2's color is [1]
vertex3's color is [1]
vertex4's color is [1]

H_A = 0.0
H_B = 0.30000000000000004
H = 0.30000000000000004
[1, 0, 1, 0, 1

When we have $H = 0$ these as answers.